In [6]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc

In [8]:
def normalize_adata(adata, hvg_genes=2000):
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes=hvg_genes)
    if hvg_genes is not None:
        return adata[:,adata.var["highly_variable"]]
    return adata

In [22]:
data_name="pancreatic"
DATA_PATH="./"
SAVE_PATH="./"

In [23]:
if data_name=="zebrafish":
    counts = pd.read_csv(f"{DATA_PATH}/three_interpolation-count_data-hvg.csv", header=0, index_col=0)
    meta = pd.read_csv(f"{DATA_PATH}/meta_data.csv", header=0, index_col=0)
    heldout_tp = [5.3,7,9]
    adata=ad.AnnData(counts.values, obs=pd.DataFrame(counts.index, index=counts.index, columns=["cells"]), var =pd.DataFrame(counts.columns,index=counts.columns, columns=["gene"]))
    adata.obs["time"] = meta["HPF"].to_list()

elif data_name=="EMT":
    counts = pd.read_csv(f"{DATA_PATH}/count_data.csv", index_col=0).T
    meta=pd.read_csv(f"{DATA_PATH}/meta_data.csv", index_col=0)
    heldout_tp = ["1d"]
    adata=ad.AnnData(counts.values, obs=pd.DataFrame(counts.index, index=counts.index, columns=["cells"]), var =pd.DataFrame(counts.columns,index=counts.columns, columns=["gene"]))
    adata.obs["time"] = meta["x"].to_list()
    adata = adata[adata.obs["time"].isin(['3d', '7d', '1d','8h', '0d'])]

elif data_name=="pancreatic":
    counts = pd.read_csv(f"{DATA_PATH}/data",sep="\t", index_col="# library.barcode")
    meta=pd.read_csv(f"{DATA_PATH}/GSE114412_Stage_5.all.cell_metadata.tsv", index_col="library.barcode", sep="\t")
    heldout_tp=[3,6]
    adata=ad.AnnData(counts.values, obs=pd.DataFrame(counts.index, index=counts.index, columns=["cells"]), var =pd.DataFrame(counts.columns,index=counts.columns, columns=["gene"]))
    adata.obs["time"] = meta["CellWeek"].to_list()

In [24]:
total_timepoints = len(adata.obs["time"].unique())

In [25]:
adata_sub = adata[~adata.obs["time"].isin(heldout_tp)]
adata_sub = normalize_adata(adata_sub)
adata_he = adata[adata.obs["time"].isin(heldout_tp)]
adata_he = normalize_adata(adata_he, hvg_genes=None)
adata_he = adata_he[:,adata_sub.var.index]

df = pd.DataFrame(np.concatenate((adata_sub.X, adata_he.X)), columns=adata_sub.var.index)
df.index = adata_sub.obs.index.tolist()+adata_he.obs.index.tolist()
df["time"] = adata_sub.obs["time"].to_list()+ adata_he.obs["time"].to_list()

# if data_name=="EMT":
#     timepoint_map={"0d":0,"8h":1,"1d":2,"3d":3,"7d":4}
# else:
#     timepoint_map={k:v for k,v in zip(np.arange(total_timepoints), sorted(adata.obs["time"].unique()))}
#     rev_timepoint_map={k:v for k,v in zip(df["time"].unique(), np.arange(total_timepoints))}
# df["time"] = df["time"].map(rev_timepoint_map)

In [13]:
# df.to_csv(f"{SAVE_PATH}/preprocessed_data.csv")